В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням алгоритмів бустингу: XGBoost та LightGBM, а також використаємо бібліотеку HyperOpt для оптимізації гіперпараметрів.

## 0. Read the data
Зчитайте дані `train.csv` в змінну `raw_df` та скористайтесь наведеним кодом нижче аби розділити дані на трнувальні та валідаційні і розділити дані на ознаки з матириці Х та цільову змінну. Назви змінних `train_inputs, train_targets, train_inputs, train_targets` можна змінити на ті, які Вам зручно.

  Наведений скрипт - частина отриманого мною скрипта для обробки даних. Ми тут не викнуємо масштабування та обробку категоріальних змінних, бо хочемо це делегувати алгоритмам, які будемо використовувати. Якщо щось не розумієте в наведених скриптах, рекомендую розібратись: навичка читати код - важлива складова роботи в машинному навчанні.

In [ ]:
import pandas as pd
import numpy as np
import cudf

from sklearn.base import BaseEstimator
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from typing import Tuple
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
def split_train_val(df: pd.DataFrame,
                    target_col: str,
                    test_size: float = 0.2,
                    random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split the dataframe into training and validation sets.

    Args:
        df (pd.DataFrame): The raw dataframe.
        target_col (str): The target column for stratification.
        test_size (float): The proportion of the dataset to include in the validation split.
        random_state (int): Random state for reproducibility.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: Training and validation dataframes.
    """
    train_df, val_df = train_test_split(df,
                                        test_size=test_size,
                                        random_state=random_state,
                                        stratify=df[target_col])
    return train_df, val_df


def separate_inputs_targets(df: pd.DataFrame,
                            input_cols: list,
                            target_col: str) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Separate inputs and targets from the dataframe.

    Args:
        df (pd.DataFrame): The dataframe.
        input_cols (list): List of input columns.
        target_col (str): Target column.

    Returns:
        Tuple[pd.DataFrame, pd.Series]: DataFrame of inputs and Series of targets.
    """
    inputs = df[input_cols].copy()
    targets = df[target_col].copy()
    return inputs, targets


def compute_auroc(
    model: BaseEstimator,
    inputs: pd.DataFrame,
    targets: pd.Series
) -> float:
    """
    Computes the Area Under the Receiver Operating Characteristic Curve (AUROC)
    for a given dataset.

    Args:
        model (BaseEstimator): A trained classification model that supports
            probability predictions via `predict_proba`.
        inputs (Union[pd.DataFrame, np.ndarray]): Feature matrix for prediction.
        targets (Union[pd.Series, np.ndarray]): True target labels.

    Returns:
        float: AUROC score for the given dataset.
    """
    predict_y = model.predict_proba(inputs)[:, 1]
    auroc = roc_auc_score(targets, predict_y)

    return auroc

In [3]:
RANDOM_STATE = 42

In [4]:
raw_df = pd.read_csv('train.csv', index_col=0)
input_cols = list(raw_df.columns)[2:-1]
target_col = 'Exited'
train_df, val_df = split_train_val(raw_df,
                                   target_col,
                                   test_size=0.2,
                                   random_state=RANDOM_STATE)

train_inputs, train_targets = separate_inputs_targets(train_df, input_cols, target_col)
val_inputs, val_targets = separate_inputs_targets(val_df, input_cols, target_col)

## 1. Categorical type
В тренувальному та валідаційному наборі перетворіть категоріальні ознаки на тип `category`. Можна це зробити двома способами:
 1. `df[col_name].astype('category')`, як було продемонстровано в лекції
 2. використовуючи метод `pd.Categorical(df[col_name])`

In [5]:
cat_features = train_inputs.select_dtypes(include='object').columns
train_inputs[cat_features] = train_inputs[cat_features].astype('category')
val_inputs[cat_features] = val_inputs[cat_features].astype('category')

In [6]:
train_inputs.dtypes

,0
CreditScore,float64
Geography,category
Gender,category
Age,float64
Tenure,float64
Balance,float64
NumOfProducts,float64
HasCrCard,float64
IsActiveMember,float64
EstimatedSalary,float64


## 2. `XGBoostClassifier`
Навчіть на отриманих даних модель `XGBoostClassifier`. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів XGBoostClassifier - тут https://xgboost.readthedocs.io/en/stable/parameter.html#global-config

  **Важливо:** зробіть такі налаштування `XGBoostClassifier` аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Можна також, якщо працюєте в Google Colab, увімкнути можливість використання GPU (`Runtime -> Change runtime type -> T4 GPU`) і встановити параметр `device='cuda'` в `XGBoostClassifier` для пришвидшення тренування бустинг моделі.
  
  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням DecisionTrees раніше. Чи вийшло покращити якість?

Convert inputs into CUDA DataFrame for matche the device `cuda`

In [7]:
train_inputs_cuda = cudf.DataFrame(train_inputs)
val_inputs_cuda = cudf.DataFrame(val_inputs)

In [ ]:
xgb_clf = XGBClassifier(
    n_estimators=50,
    max_depth=3,
    grow_policy='lossguide',
    learning_rate=1,
    verbosity=1,
    objective='binary:logistic',
    random_state=RANDOM_STATE,
    missing=np.nan,
    device='cuda',
    enable_categorical=True,
)

xgb_clf.fit(train_inputs_cuda,
            train_targets,
            eval_set=[(val_inputs_cuda, val_targets)],
            verbose=False)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy='lossguide', importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [9]:
train_auroc = compute_auroc(xgb_clf, train_inputs_cuda, train_targets)
val_auroc = compute_auroc(xgb_clf, val_inputs_cuda, val_targets)

print(f"AUROC on Train: {train_auroc:.3f}")
print(f"AUROC on Validation: {val_auroc:.3f}")


AUROC on Train: 0.958
AUROC on Validation: 0.927


This model tends to overfit, but perform better than `DesionTreeClassifier` (0.916) or `LogisticRegression` (0.87) without `PolynomialFeatures`.
Similar result we had only for tuned `LogisticRegression` with `PolynomialFeatures(degree=5)`.

## 3. `Hyperopt` for `XGBoostClassifier`
Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `XGBoostClassifier` з лекції знайдіть оптимальні значення гіперпараметрів `XGBoostClassifier` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **20**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. В ній ми маємо задати loss - це може будь-яка метрика, але бажано використовувтаи ту, яка цільова в вашій задачі. Чим менший лосс - тим ліпша модель на думку hyperopt. Тож, тут нам треба задати loss - негативне значення AUROC. В лекції ми натомість використовували Accuracy.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_clf` модель `XGBoostClassifier` з найкращими гіперпараметрами
    - навчіть модель `final_clf`
    - оцініть якість моделі `final_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (2) цього завдання?

In [ ]:
%%time
def objective(space):
    clf = XGBClassifier(early_stopping_rounds=10,
                        **space)

    clf.fit(
        train_inputs_cuda,
        train_targets,
        eval_set=[(val_inputs_cuda, val_targets)],
        verbose=False)

    auroc = compute_auroc(clf, val_inputs_cuda, val_targets)

    return {'loss': -auroc, 'status': STATUS_OK}


space = {
    'n_estimators':     hp.choice("n_estimators", np.arange(50, 500, 25, dtype='int')),
    'max_depth':        hp.choice('max_depth', np.arange(2, 20, 1, dtype=int)),
    'learning_rate':    hp.uniform('learning_rate', 0.01, 1),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 50, 1, dtype=int)),
    'subsample':        hp.uniform('subsample', 0.5, 1.),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.),
    'gamma':            hp.uniform('gamma', 0, 10),
    'reg_alpha':        hp.uniform('reg_alpha', 0, 1),
    'reg_lambda':       hp.uniform('reg_lambda', 0, 1),
    'random_state':     RANDOM_STATE,
    'missing':          np.nan,
    'enable_categorical': True,
    'device':           'cuda',
    'eval_metric':      'auc',
    'grow_policy':      'lossguide',
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

print("The best hyperparameters:", best)

100%|██████████| 20/20 [00:02<00:00,  7.41trial/s, best loss: -0.9363944715001029]
The best hyperparameters: {'colsample_bytree': 0.7995781279469161, 'gamma': 5.9622437112952404, 'learning_rate': 0.12237154039949696, 'max_depth': 13, 'min_child_weight': 9, 'n_estimators': 6, 'reg_alpha': 0.6059658274317766, 'reg_lambda': 0.020878773201130074, 'subsample': 0.502354857245574}
CPU times: user 2.87 s, sys: 85.8 ms, total: 2.96 s
Wall time: 2.71 s


In [11]:
best_params = space_eval(space, best)

In [16]:
final_clf = XGBClassifier(**best_params)
final_clf.fit(train_inputs_cuda, train_targets)

train_auroc = compute_auroc(final_clf, train_inputs_cuda, train_targets)
val_auroc = compute_auroc(final_clf, val_inputs_cuda, val_targets)

print(f"AUROC on Train: {train_auroc:.4f}")
print(f"AUROC on Validation: {val_auroc:.4f}")

AUROC on Train: 0.9408
AUROC on Validation: 0.9367


This model perform better than `XGBClassifier` model without hyperparameters tuning and has the best result among all my previous attempts.

## 4. `LightGBM`
Навчіть на наших даних модель LightGBM. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів LightGBM - тут https://lightgbm.readthedocs.io/en/latest/Parameters.html

  **Важливо:** зробіть такі налаштування LightGBM аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Аби передати категоріальні колонки в LightGBM - необхідно виявити їх індекси і передати в параметрі `cat_feature=cat_feature_indexes`

  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням XGBoostClassifier раніше. Чи вийшло покращити якість?

In [13]:
cat_features_idx = [train_inputs.columns.get_loc(col) for col in cat_features]

In [14]:
lgb_clf = LGBMClassifier(
    max_depth=3,
    learning_rate=0.1,
    n_estimators=50,
    random_state=RANDOM_STATE,
    verbosity=-1,
)

lgb_clf.fit(train_inputs,
            train_targets,
            eval_set=[(val_inputs, val_targets)],
            eval_metric='auc',
            categorical_feature=cat_features_idx)

LGBMClassifier(max_depth=3, n_estimators=50, random_state=42, verbosity=-1)

In [17]:
train_auroc = compute_auroc(lgb_clf, train_inputs, train_targets)
val_auroc = compute_auroc(lgb_clf, val_inputs, val_targets)

print(f"AUROC on Train: {train_auroc:.4f}")
print(f"AUROC on Validation: {val_auroc:.4f}")

AUROC on Train: 0.9388
AUROC on Validation: 0.9362


The `LGBMClassifier` perform well, like `XGBClassifier` after hyperparameters tuning.

## 5. `Hyperopt` for `LightGBM`
Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `LightGBM` з лекції знайдіть оптимальні значення гіперпараметрів `LightGBM` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **10**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. І тут ми також ставимо loss - негативне значення AUROC, як і при пошуці гіперпараметрів для XGBoost. До речі, можна спробувати написати код так, аби в objective передавати лише модель і не писати схожий код двічі :)

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_lgb_clf` модель `LightGBM` з найкращими гіперпараметрами
    - навчіть модель `final_lgb_clf`
    - оцініть якість моделі `final_lgb_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (4) цього завдання?

In [ ]:
%%time
def objective(space):
    clf = LGBMClassifier(early_stopping_rounds=10,
                         **space)

    clf.fit(
        train_inputs,
        train_targets,
        eval_set=[(val_inputs, val_targets)])

    auroc = compute_auroc(clf, val_inputs, val_targets)

    return {'loss': -auroc, 'status': STATUS_OK}


space = {
    'max_depth':        hp.choice('max_depth', np.arange(2, 20, 1, dtype=int)),
    'num_leaves':       hp.choice('num_leaves', np.arange(20, 150, 1, dtype=int)),
    'learning_rate':    hp.uniform('learning_rate', 0.01, 1),
    'n_estimators':     hp.choice("n_estimators", np.arange(50, 500, 25, dtype='int')),
    'subsample':        hp.uniform('subsample', 0.5, 1.),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.),
    'min_split_gain':   hp.uniform('min_split_gain', 0, 0.1),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 50, 1, dtype=int)),
    'reg_alpha':        hp.uniform('reg_alpha', 0, 1),
    'reg_lambda':       hp.uniform('reg_lambda', 0, 1),
    'random_state':     RANDOM_STATE,
    'metric':           'auc',
    'subsample_freq':   1,
    'verbose':          -1
}

trials = Trials()

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

print("The best hyperparameters:", best)

100%|██████████| 10/10 [00:00<00:00, 10.23trial/s, best loss: -0.9345812469991082]
The best hyperparameters: {'colsample_bytree': 0.99819354317856, 'learning_rate': 0.48960896051985064, 'max_depth': 2, 'min_child_weight': 16, 'min_split_gain': 0.09802102437965353, 'n_estimators': 7, 'num_leaves': 73, 'reg_alpha': 0.3442732362945927, 'reg_lambda': 0.5940880697769494, 'subsample': 0.7222065353634037}
CPU times: user 988 ms, sys: 14.9 ms, total: 1 s
Wall time: 999 ms


In [19]:
best_params = space_eval(space, best)

final_lgb_clf = LGBMClassifier(**best_params)
final_lgb_clf.fit(train_inputs, train_targets)

train_auroc = compute_auroc(final_lgb_clf, train_inputs, train_targets)
val_auroc = compute_auroc(final_lgb_clf, val_inputs, val_targets)

print(f"AUROC on Train: {train_auroc:.4f}")
print(f"AUROC on Validation: {val_auroc:.4f}")

AUROC on Train: 0.9739
AUROC on Validation: 0.9184


The `LGBMClassifier` model performed worse after hyperparameter tuning, likely due to overfitting. Increasing the number of evaluation steps might help in better generalization.

## 6. Submission on Kaggle
Оберіть модель з експериментів в цьому ДЗ і зробіть новий `submission` на Kaggle та додайте код для цього і скріншот скора на публічному лідерборді.
  
  **Напишіть коментар, чому ви обрали саме цю модель?**

  І я вас вітаю - це останнє завдання з цим набором даних 💪 На цьому етапі корисно проаналізувати, які моделі показали себе найкраще і подумати, чому.

In [22]:
test_df = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

test_inputs = test_df[input_cols].copy()
test_inputs[cat_features] = test_inputs[cat_features].astype('category')

test_preds_proba = lgb_clf.predict_proba(test_inputs)[:, 1]

submission['Exited'] = test_preds_proba

submission.to_csv('submission_lgb_clf.csv', index=False)

<image src=https://i.imgur.com/5a37P81.png width=800>

For this submissing, I chose the `LGBMClassifier` model before tuning. Although the `XGBClassifier` performed slightly better on the validation set after tuning, it performed worse on the training set.